In [24]:
from __future__ import print_function
import pandas as pd
from google.cloud import vision
import io
import os.path
import os


In [25]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/romanbreyer/Downloads/global-timer-355314-884e9e06301f.json"

In [26]:
def label_detection(path):
    """ Label detection function - returns Dataframe with results. Always returns 10 labels"""
    # Get image in correct encoding
    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    # API Call
    client = vision.ImageAnnotatorClient()
    image = vision.Image(content = content)
    response = client.label_detection(image=image)  #Limit to 10 for consistency

    # Get file name
    file_name = os.path.basename(path)
    print("="*30)
    print(f"Label detection for {file_name}")
    # Create Dataframe for return - there are 10 labels return every time
    collect_label = pd.DataFrame()

    # Create helper list for Dataframe
    helper = [path, file_name]
    vertices_labels = {}
    # Iterate over all annotations and get labels with confidence score
    iter = 0
    for label in response.label_annotations:
        iter += 1
        #print(label.description, '(%.2f%%)' % (label.score*100.))
        helper.append(f"{label.description} ({round(label.score*100.,2)}%)")

    # check if there are 10 labels in list, otherwise fill with empty
    if len(helper) < 10:
        while len(helper) != 10:
            helper.append("no response")
    # Fill and return Dataframe
    collect_label[f"{file_name}"] = helper
    return collect_label




In [27]:
def emotion_detection(path):
    """ Face Detection function - returns DataFrame with results.  Possible Emotions: "joy, sorrow, anger, surprise")"""
    with io.open(path, 'rb') as image_file:
        content = image_file.read()
    client = vision.ImageAnnotatorClient()
    image = vision.Image(content = content)
    response = client.face_detection(image=image)
    collect_emotions = pd.DataFrame()
    file_name = os.path.basename(path)
    print("="*30)
    print(f"Emotion detection for {file_name}")
    counter = 0
    for face in response.face_annotations:
        counter += 1
        likelihood_joy = vision.Likelihood(face.joy_likelihood)
        likelihood_anger = vision.Likelihood(face.anger_likelihood)
        likelihood_sorrow = vision.Likelihood(face.sorrow_likelihood)
        likelihood_surprise = vision.Likelihood(face.surprise_likelihood)
        vertices = ['(%s,%s)' % (v.x, v.y) for v in face.bounding_poly.vertices]

        collect_emotions[f"Face {counter} in file {file_name}"] = [path, file_name, str(vertices), likelihood_joy.name, likelihood_anger.name, likelihood_sorrow.name, likelihood_surprise.name]

    return collect_emotions


In [28]:
def emotion_pipeline(limit, dir, ignore_max = False):
    """Function to shoot X amount of files in directory through Vision API and aggregate results (limit before it gets costly should be 500)
        Iterates through the given directory and processes every picture until "num" is reached, calls emotion_detection() and label_detection() in every iteration (two API Calls)"""

    # Initialize Dataframes for export
    label_fetch = pd.DataFrame({"Row Name": ["FilePath","FileName","Label_1","Label_2","Label_3","Label_4","Label_5","Label_6","Label_7","Label_8","Label_9","Label_10"]})
    emotions_fetch = pd.DataFrame({"Category":["FilePath",'FileName','Bounds','JOY','ANGER','SORROW','SURPRISE']})

    print("=" * 30)
    # For Label and Face Detection, two separate Calls are needed. Account for this in the number of API Calls!!!
    if (not ignore_max) and limit > 490:
        print("Limit will exceed max. monthly API calls - try again with ignore_max = True")
        return emotions_fetch, label_fetch
    print(f"Getting Emotion and Label Recognition, for {limit} files in {dir}")



    #Set counter and iterate over files in dir until defined limit is reached
    counter = 0
    for file in os.listdir(dir):
        counter += 1
        if counter <= limit:
            filepath = os.path.join(dir, file)
            if os.path.isfile(filepath):
                df_emotions_single = emotion_detection(filepath)
                emotions_fetch = pd.concat([emotions_fetch,df_emotions_single], axis=1)

                df_labels_single = label_detection(filepath)
                label_fetch = pd.concat([label_fetch, df_labels_single], axis = 1)
        else:
            print("Limit reached, not all pictures processed")
            return emotions_fetch, label_fetch
    print("All pictures processed, limit was not reached")
    return emotions_fetch, label_fetch


    # Aggregate all results in DataFrame by joining

In [29]:
# Function to visualize results of a specific picture and the annotations made by Vision API
def get_jpeg():
    return

In [30]:
def get_path(path):
    """Check if Directory exists, if not -> create it. Then return the path."""

    if not path[-1:] == "\\": path = path + "\\"

    path_snippets = path[:-1].split("\\")
    path_check = ""

    for snip in path_snippets:
        path_check = f"{path_check}{snip}\\"
        if not os.path.exists(path_check): os.mkdir(path_check)

    return f"{path}"

In [31]:
def export_toCSV(df, csv_path, csv_name = "Export_data.csv", index = True, index_name = "Index", header = True, sep = ","):
    """ Handles export of dataframe /w some extras"""
    # Console Output
    print(f"\nExporting data to CSV({csv_path}{csv_name}")
    # Create Path if necessary
    csv_path = get_path(csv_path)
    # Export Data
    df.index.name = index_name
    if csv_name[-4:] != ".csv":
        csv_name += ".csv"
    df.to_csv(f"{csv_path}{csv_name}", sep = sep , index = index, header =header)

In [32]:
''' Sample function call'''
emotions, labels = emotion_pipeline(10,"data/vision_input",ignore_max = True)


Getting Emotion and Label Recognition, for 10 files in data/vision_input
Emotion detection for images-2.jpeg
Label detection for images-2.jpeg
Emotion detection for test_img.jpg
Label detection for test_img.jpg
All pictures processed, limit was not reached


In [33]:
# Export File
export_toCSV(emotions,csv_path= "data/vision_output/", csv_name="Emotions_Analysis.csv" )


Exporting data to CSV(data/vision_output/Emotions_Analysis.csv


In [34]:
# Export File
export_toCSV(labels,csv_path= "data/vision_output/", csv_name="Labels_Analysis.csv" )


Exporting data to CSV(data/vision_output/Labels_Analysis.csv


In [35]:
emotions

,Category,Face 1 in file test_img.jpg
Index,,
0,FilePath,data/vision_input/test_img.jpg
1,FileName,test_img.jpg
2,Bounds,"['(726,134)', '(1307,134)', '(1307,810)', '(72..."
3,JOY,VERY_LIKELY
4,ANGER,VERY_UNLIKELY
5,SORROW,VERY_UNLIKELY
6,SURPRISE,VERY_UNLIKELY


In [36]:
labels

,Row Name,images-2.jpeg,test_img.jpg
Index,,,
0,FilePath,data/vision_input/images-2.jpeg,data/vision_input/test_img.jpg
1,FileName,images-2.jpeg,test_img.jpg
2,Label_1,Dog (96.29%),Forehead (98.47%)
3,Label_2,Plant (95.04%),Smile (98.21%)
4,Label_3,Dog breed (90.05%),Cheek (97.9%)
5,Label_4,Carnivore (89.59%),Sleeve (85.43%)
6,Label_5,Companion dog (81.98%),Collar (81.93%)
7,Label_6,Grass (79.03%),Spokesperson (79.33%)
8,Label_7,Happy (76.68%),Happy (77.44%)
